<a href="https://colab.research.google.com/github/Subhutii/GRIT-AI/blob/main/textRank_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textacy

In [3]:
from typing import List, Tuple
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from textacy import extract, make_spacy_doc

import spacy
from spacy.lang.ja.examples import sentences 

#https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html


In [4]:
pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.6 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172961 sha256=60d0a73660cec59e16355f5d969356378091cab7b97311ce35ad8d6a5d1fd0b0
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ja_core_news_sm

In [6]:
import pandas as pd
import numpy as np
import neologdn
import re
import string
from typing import List, Tuple
import spacy
from spacy.lang.ja.examples import sentences
from textacy import extract


In [7]:
# テキスト取得
with open("text_ja.txt", "r") as file:
    data = file.read().replace("\n", "")
article = data.replace(u"\xa0", u" ") # ノーブレークスペース削除


In [8]:
print

<function print>

ストップワード処理


In [9]:
# 前処理
def preprocess(x):
  #絵文字の除去
  emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"  # emoticons
            "\U0001F300-\U0001F5FF"  # symbols & pictographs
            "\U0001F680-\U0001F6FF"  # transport & map symbols
            "\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "]+",
            flags=re.UNICODE,
        )
  x = emoji_pattern.sub(r"", x)

  x = neologdn.normalize(x)
  x = x.lower()
  x = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", x)#URLの除去
  x = re.sub(r"[!-/:-@[-`{-~]", r" ", x)# 半角記号の置換
  x = re.sub("[■-♯]", " ", x)# 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
  # x = re.sub(r"(\d)([,.])(\d+)", "\1\3", x)#桁区切り (+ 小数点) 除去
  # x = re.sub(r"\d+", "0", x)#数字を0に置換
  # x = re.sub(r"・", ", ", x)
  x = re.sub(r"[\(\)「」【】]", "", x)# 括弧削除（まだ消すやつある）

  return x


spacy(textacy)+textrank

In [10]:
#extract後の可視化
def decompose_keyterms(keyterm_list: List[str]) -> Tuple:#list [], tuple ()  変更ができるのがリスト。変更ができないのがタプル
#keyterm_listをelに入れて el[0].replace("\n", " ") で置き換えたもの（改行を""に置き換え，0番目の要素）をtermsに入れる
    # terms=[]
    # for el in keyterm_list:
    #     terms.append(el[0])
    
    terms = [el[0] for el in keyterm_list]
    # terms = [el[0].replace("\n", " ") for el in keyterm_list]
    scores = [el[1] for el in keyterm_list]
    # scores = np.asarray([el[1] for el in keyterm_list])
    return terms, scores

#トークン化
def create_doc(article):
    texts = article.split(',')
    article_1 = ""
    for text in texts:
      article_1+=text+","

    # doc = make_spacy_doc(article_1, lang="en_core_web_sm")
    nlp = spacy.load("ja_core_news_sm")
    doc = nlp(article_1)
    return doc

#textrank
def textrank_algo(doc):
    textrank = extract.keyterms.textrank(doc, normalize="lemma")
    terms_textrank, scores_textrank = decompose_keyterms(textrank)

    return terms_textrank, scores_textrank

In [11]:
#yake
def yake_algo(doc):
    yake = extract.keyterms.yake(doc, normalize="lemma")
    terms_yake, scores_yake = decompose_keyterms(yake)

    return terms_yake, scores_yake

#scake
def scake_algo(doc):
    scake = extract.keyterms.scake(doc, normalize="lemma")
    terms_scake, scores_scake = decompose_keyterms(scake)

    return terms_scake, scores_scake

#sgrank
def sgrank_algo(doc):
    sgrank = extract.keyterms.sgrank(doc, normalize="lemma")
    terms_sgrank, scores_sgrank = decompose_keyterms(sgrank)
    return terms_sgrank, scores_sgrank

In [12]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [15]:
#呼び出し
def main(article):
  #part1
    text = preprocess(article) #ストップワード削除
    doc = create_doc(text)

#part2
    # doc = create_doc(article)

    terms_textrank, score_textrank = textrank_algo(doc)
    # terms_yake, score_yake = yake_algo(doc)
    # terms_scake, score_scake = scake_algo(doc)
    # terms_sgrank, scores_sgrank = sgrank_algo(doc)

    print("textrank", terms_textrank)
    # print("textrank score", score_textrank)
    # print("yake", terms_yake)
    # print("scake", terms_scake)
    # print("sgrank", terms_sgrank)
    print(type(terms_textrank))

    #品詞表示
    # POS_tag = nltk.pos_tag(terms_textrank)
    # print(POS_tag)

main(article)

textrank ['富士 山 本宮 浅間 大社', '笑 富士 山', '富士 山 パーキング', '合 目 着', '難しい 山', '山 小屋', '合 目 あたり', '問題 ない', '富士 急行 バス', '持ち主 徳川 家康']
<class 'list'>
